In [2]:
extracted_dir = '/content/drive/MyDrive/ECG Report Reader/ECG_Datasets/Data sets'

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the image data generator with augmentation
datagen = ImageDataGenerator(
    rescale=1./255,        # Normalize the images
    validation_split=0.2   # Split data into training and validation sets
)

# Load the training data
train_generator = datagen.flow_from_directory(
    extracted_dir,          # Path to the extracted directory
    target_size=(224, 224),
    color_mode='grayscale',
    batch_size=32,
    class_mode='sparse',    # Use 'sparse' for integer labels
    subset='training'       # Set as training data
)

# Load the validation data
validation_generator = datagen.flow_from_directory(
    extracted_dir,          # Path to the extracted directory
    target_size=(224, 224),
    color_mode='grayscale',
    batch_size=32,
    class_mode='sparse',    # Use 'sparse' for integer labels
    subset='validation'     # Set as validation data
)


Found 427 images belonging to 3 classes.
Found 105 images belonging to 3 classes.


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def create_model(input_shape):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(3, activation='softmax')  # 3 classes: Normal, Abnormal, Highly Emergency
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

input_shape = (224, 224, 1)
model = create_model(input_shape)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [19]:
# Train the model
history = model.fit(
    train_generator,
    epochs=100,# Number of epochs
    validation_data=validation_generator,
    steps_per_epoch=train_generator.samples // 32,
    validation_steps=validation_generator.samples // 32
)

# Save the model
model.save('ecg_classifier_model.h5')


Epoch 1/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 63s 4s/step - accuracy: 1.0000 - loss: 2.1537e-05 - val_accuracy: 1.0000 - val_loss: 0.0031
Epoch 2/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 1.0000 - loss: 1.0963e-05 - val_accuracy: 1.0000 - val_loss: 0.0206
Epoch 3/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 75s 4s/step - accuracy: 1.0000 - loss: 1.5200e-05 - val_accuracy: 1.0000 - val_loss: 0.0021
Epoch 4/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 1.0000 - loss: 1.6670e-05 - val_accuracy: 1.0000 - val_loss: 7.2872e-05
Epoch 5/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 77s 4s/step - accuracy: 1.0000 - loss: 2.0530e-05 - val_accuracy: 1.0000 - val_loss: 0.0055
Epoch 6/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 1.0000 - loss: 4.0978e-06 - val_accuracy: 1.0000 - val_loss: 4.7683e-06
Epoch 7/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 60s 4s/step - accuracy: 1.0000 - loss: 1.5062e-05 - val_accuracy: 1.0000 - val_loss: 0.0034
Epoch 8/100
13/13 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - accuracy: 1.0000 -

In [6]:
# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Loss: {loss}')
print(f'Validation Accuracy: {accuracy}')


4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 1.0000 - loss: 0.0043
Validation Loss: 0.0034414927940815687
Validation Accuracy: 1.0


In [7]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

In [8]:
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    resized_image = cv2.resize(image, (224, 224))
    normalized_image = resized_image / 255.0
    expanded_image = np.expand_dims(normalized_image, axis=-1)
    expanded_image = np.expand_dims(expanded_image, axis=0)
    return expanded_image

def classify_ecg(image_path, model_path='ecg_classifier_model.h5'):
    model = load_model(model_path)
    processed_image = load_and_preprocess_image(image_path)
    prediction = model.predict(processed_image)
    class_names = ['Normal', 'Abnormal', 'Highly Emergency']
    predicted_class = class_names[np.argmax(prediction)]
    return predicted_class

In [18]:
# Example usage
result = classify_ecg('/content/drive/MyDrive/ECG Report Reader/ecg.jpg')
print(f'The ECG classification result is: {result}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
The ECG classification result is: Abnormal
